In [1]:
# This is a script to generate nice readable tables for the identified clusters 
# Import libraries
import sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle

from astropy.io import ascii
from astropy.table import Table


import warnings
warnings.filterwarnings("once")

In [2]:
# Import data
cm_vel_all = pd.read_hdf('../data/dr3_near_vel_plxzp_g2_only.h5')
orb_param_all = pd.read_hdf('../data/dr3_orb_param_err_g2_only.h5')

/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
# Set all -9999 values in APOGEE and RAVE6 cnn [Fe/H] and alpha to np.nan for easier calculation later
ind_ap_999 = np.where(cm_vel_all.loc[:,'m_h_ap'] < -100)[0]
ind_r6c_999 = np.where(cm_vel_all.loc[:,'m_h_r6c'] < -100)[0]
ind_ap17_999 = np.where(cm_vel_all.loc[:,'m_h_ap17'] < -100)[0]
cm_vel_all.loc[ind_ap_999,'m_h_ap'] = np.nan
cm_vel_all.loc[ind_r6c_999,'m_h_r6c'] = np.nan
cm_vel_all.loc[ind_ap17_999,'m_h_ap17'] = np.nan

ind_ap_999 = np.where(cm_vel_all.loc[:,'m_h_err_ap'] < -100)[0]
ind_ap17_999 = np.where(cm_vel_all.loc[:,'m_h_err_ap17'] < -100)[0]
ind_r6c_999 = np.where(cm_vel_all.loc[:,'m_h_err_r6c'] < -100)[0]
ind_l6s_999 = np.where(cm_vel_all.loc[:,'m_h_err_l6s'] < -100)[0]
cm_vel_all.loc[ind_ap_999,'m_h_err_ap'] = np.nan
cm_vel_all.loc[ind_ap17_999,'m_h_err_ap17'] = np.nan
cm_vel_all.loc[ind_r6c_999,'m_h_err_r6c'] = np.nan
cm_vel_all.loc[ind_l6s_999,'m_h_err_l6s'] = np.nan

ind_ap_999 = np.where(cm_vel_all.loc[:,'alpha_m_ap'] < -100)[0]
ind_ap17_999 = np.where(cm_vel_all.loc[:,'alpha_m_ap17'] < -100)[0]
ind_r6c_999 = np.where(cm_vel_all.loc[:,'alpha_m_r6c'] < -100)[0]
cm_vel_all.loc[ind_ap_999,'alpha_m_ap'] = np.nan
cm_vel_all.loc[ind_ap17_999,'alpha_m_ap17'] = np.nan
cm_vel_all.loc[ind_r6c_999,'alpha_m_r6c'] = np.nan

ind_ap_999 = np.where(cm_vel_all.loc[:,'alpha_m_err_ap'] < -100)[0]
ind_ap17_999 = np.where(cm_vel_all.loc[:,'alpha_m_err_ap17'] < -100)[0]
ind_r6c_999 = np.where(cm_vel_all.loc[:,'alpha_m_err_r6c'] < -100)[0]
ind_l6s_999 = np.where(cm_vel_all.loc[:,'alpha_m_err_l6s'] < -100)[0]
cm_vel_all.loc[ind_ap_999,'alpha_m_err_ap'] = np.nan
cm_vel_all.loc[ind_ap17_999,'alpha_m_err_ap17'] = np.nan
cm_vel_all.loc[ind_r6c_999,'alpha_m_err_r6c'] = np.nan
cm_vel_all.loc[ind_l6s_999,'alpha_m_err_l6s'] = np.nan

In [4]:
# Count the number of metallicity matches from each survey
datasets = ['ap17','r6c','l6s']
for dataset in datasets:
    N = len(np.where(np.isnan(cm_vel_all.loc[:,f'm_h_{dataset}']) == False)[0])
    print(f'Dataset {dataset} has {N} matches for metallicity.')
    
    N = len(np.where(np.isnan(cm_vel_all.loc[:,f'alpha_m_{dataset}']) == False)[0])
    print(f'Dataset {dataset} has {N} matches for alpha.')

Dataset ap17 has 211798 matches for metallicity.
Dataset ap17 has 211788 matches for alpha.
Dataset r6c has 293596 matches for metallicity.
Dataset r6c has 293649 matches for alpha.
Dataset l6s has 632223 matches for metallicity.
Dataset l6s has 0 matches for alpha.


In [5]:
# calculate the columns for the scaled action diamond
orb_param_all['Jtot'] = np.sqrt(orb_param_all['Jphi']**2+orb_param_all['JR']**2+orb_param_all['Jz']**2)
orb_param_all['diamond_x']=orb_param_all['Jphi']/orb_param_all['Jtot']
orb_param_all['diamond_y']=(orb_param_all['Jz']-orb_param_all['JR'])/orb_param_all['Jtot']

# Calculate the error for Jtot and diamond_x/y
orb_param_all['e_Jtot'] = np.sqrt(orb_param_all['Jphi']**2*orb_param_all['e_Jphi']**2+orb_param_all['JR']**2*orb_param_all['e_JR']**2+orb_param_all['Jz']**2*orb_param_all['e_Jz']**2)/orb_param_all['Jtot']
orb_param_all['e_diamond_x']=np.sqrt(orb_param_all['Jtot']**2*orb_param_all['e_Jphi']**2+orb_param_all['e_Jtot']**2*orb_param_all['Jphi']**2)/orb_param_all['Jtot']**2
orb_param_all['e_diamond_y']=np.sqrt(orb_param_all['Jtot']**2*(orb_param_all['e_JR']**2+orb_param_all['e_Jz']**2)+orb_param_all['e_Jtot']**2*(orb_param_all['Jz']-orb_param_all['JR'])**2)/orb_param_all['Jtot']**2

# Calculate L_perp for clustering
orb_param_all['Lperp'] = np.sqrt(orb_param_all['Lx']**2+orb_param_all['Ly']**2)
orb_param_all['e_Lperp'] = np.sqrt(orb_param_all['Lx']**2*orb_param_all['e_Lx']**2+orb_param_all['Ly']**2*orb_param_all['e_Ly']**2)/orb_param_all['Lperp']

# This mean metallicity below is only used in the case when we actually want to cluster in or with a prior cut on metallicity
# To avoid systematic differences between spectroscopic surveys; don't mix metallicities from different surveys
# cm_vel_all['m_h_mean'] = np.nanmean(cm_vel_all[['m_h_ap17','m_h_l6s','m_h_r6c','m_h_gl3']].values, axis=1).T
# cm_vel_all['m_h_mean'], cm_vel_all['e_m_h_mean'] = cm_vel_all['m_h_ap17'], cm_vel_all['m_h_err_ap17'] 
# cm_vel_all['m_h_mean'], cm_vel_all['e_m_h_mean'] = cm_vel_all['m_h_l6s'], cm_vel_all['m_h_err_l6s'] 

In [6]:
# Import the source_id of the identified clusters
# space = 'E_act'
# space_e = 'act_etot'
space = 'vel_cyl'
space_e = 'vel_cyl'
cut = '2500'
pcut = '20'

feh_from = 'l6s'
# feh_from = 'ap17'

# Use LAMOST or APOGEE for Fe/H
cm_vel_all['m_h_mean'], cm_vel_all['e_m_h_mean'] = cm_vel_all[f'm_h_{feh_from}'], cm_vel_all[f'm_h_err_{feh_from}'] 

data_dir = './' + space + '_' + cut + '_data_20_20_err_nocut/'
extratext = '_0412_Nstackcut_' + pcut + '_' + space_e + '_hdbscan_min_samples_20_min_clustsize_20_leaf_baseline'

cluster_id = np.load(data_dir + 'member_gedr3id' + extratext + '.npy',allow_pickle=True)
# cluster_ind = np.load(data_dir + 'member_mask' + extratext + '.npy',allow_pickle=True)
# cluster_col = np.load(data_dir + 'color' + extratext + '.npy',allow_pickle=True)
# cluster_nstar = np.load(data_dir + 'member_nstar' + extratext + '.npy',allow_pickle=True)
# cluster_means = np.load(data_dir + 'member_means' + extratext + '.npy',allow_pickle=True)
# cluster_dispersions = np.load(data_dir + 'member_disps' + extratext + '.npy',allow_pickle=True)

In [7]:
# count how many clusters are identified. Note the last one is always noise
print("Number of identified clusters:",len(cluster_id)-1)

Number of identified clusters: 23


In [14]:
# Create a dataframe to hold all the output
df_output = pd.DataFrame({"gaia_edr3_sourceid": cluster_id[0], 
                          "space": ['velocity']*len(cluster_id[0]), 
                          "cluster": ['0']*len(cluster_id[0])})

for i in range(1,len(cluster_id)-1):
    source_id_tmp = cluster_id[i]
    df_tmp = pd.DataFrame({"gaia_edr3_sourceid": source_id_tmp, 
                           "space": ['velocity']*len(source_id_tmp), 
                           "cluster": [f"{i}"]*len(source_id_tmp)})
    df_output = pd.concat([df_output,df_tmp])

In [16]:
space = 'E_act'
space_e = 'act_etot'
# space = 'vel_cyl'
# space_e = 'vel_cyl'
cut = '2500'
pcut = '20'

data_dir = './' + space + '_' + cut + '_data_20_20_err_nocut/'
extratext = '_0412_Nstackcut_' + pcut + '_' + space_e + '_hdbscan_min_samples_20_min_clustsize_20_leaf_baseline'

cluster_id = np.load(data_dir + 'member_gedr3id' + extratext + '.npy',allow_pickle=True)

In [17]:
# count how many clusters are identified. Note the last one is always noise
print("Number of identified clusters:",len(cluster_id)-1)

Number of identified clusters: 8


In [18]:
for i in range(0,len(cluster_id)-1):
    source_id_tmp = cluster_id[i]
    df_tmp = pd.DataFrame({"gaia_edr3_sourceid": source_id_tmp, 
                           "space": ['iom']*len(source_id_tmp), 
                           "cluster": [f"{i}"]*len(source_id_tmp)})
    df_output = pd.concat([df_output,df_tmp])

In [19]:
df_output

,gaia_edr3_sourceid,space,cluster
0,634469512611911168,velocity,0
1,1277708416434956288,velocity,0
2,2798457105022823296,velocity,0
3,2831494371421184512,velocity,0
4,2910503176753011840,velocity,0
...,...,...,...
101,6574821393682814208,iom,7
102,6712347685314238208,iom,7
103,6895604357762028928,iom,7
104,6910944366035861376,iom,7


In [20]:
# Add the Merged cluster assignment columns
df_output['merged_cluster'] = ''

In [21]:
df_output

,gaia_edr3_sourceid,space,cluster,merged_cluster
0,634469512611911168,velocity,0,
1,1277708416434956288,velocity,0,
2,2798457105022823296,velocity,0,
3,2831494371421184512,velocity,0,
4,2910503176753011840,velocity,0,
...,...,...,...,...
101,6574821393682814208,iom,7,
102,6712347685314238208,iom,7,
103,6895604357762028928,iom,7,
104,6910944366035861376,iom,7,


In [22]:
vel_merge = ['I','II','III','IV','V','VI','VII','VIII','IX','X','XI','XII']
vel_compo = [['0'],['1'],['7','8'],['2'],['3'],['9', '11', '12', '14', '15', '17', '21', '22'],
            ['10'],['13'],['18'],['19','20'],['16'],['4','5','6']]

iom_merge = ['I','II','III','IV']
iom_compo = [['0'],['1','2'],['3'],['4','5','6','7']]

print(len(vel_merge),len(vel_compo),len(iom_merge),len(iom_compo))

12 12 4 4


In [28]:
for i in range(len(vel_merge)):
    for j in range(len(vel_compo[i])):
        mask = (df_output['space'] == 'velocity') & (df_output['cluster'] == vel_compo[i][j])
        df_output.loc[mask,'merged_cluster'] = vel_merge[i]
               
for i in range(len(iom_merge)):
    for j in range(len(iom_compo[i])):
        mask = (df_output['space'] == 'iom') & (df_output['cluster'] == iom_compo[i][j])
        df_output.loc[mask,'merged_cluster'] = iom_merge[i]

In [29]:
df_output

,gaia_edr3_sourceid,space,cluster,merged_cluster
0,634469512611911168,velocity,0,I
1,1277708416434956288,velocity,0,I
2,2798457105022823296,velocity,0,I
3,2831494371421184512,velocity,0,I
4,2910503176753011840,velocity,0,I
...,...,...,...,...
101,6574821393682814208,iom,7,IV
102,6712347685314238208,iom,7,IV
103,6895604357762028928,iom,7,IV
104,6910944366035861376,iom,7,IV


In [31]:
df_output.loc[100,:]

,gaia_edr3_sourceid,space,cluster,merged_cluster
100,4699467938708570368,velocity,9,VI
100,2638635393344081152,velocity,10,VII
100,4689639129397969536,velocity,18,IX
100,4525152705278082432,iom,3,III
100,1916383694166326656,iom,5,IV
100,6348124086267699840,iom,6,IV
100,6541189115398850560,iom,7,IV


In [32]:
df_output.to_csv('clustering_result_tables/robust_cluster_source_id.csv',index=False)

In [33]:
df_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2320 entries, 0 to 105
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   gaia_edr3_sourceid  2320 non-null   int64 
 1   space               2320 non-null   object
 2   cluster             2320 non-null   object
 3   merged_cluster      2320 non-null   object
dtypes: int64(1), object(3)
memory usage: 170.6+ KB


In [2]:
# Also export the table in MRT format using astropy 
tab = ascii.read('clustering_result_tables/robust_cluster_source_id.csv')

In [3]:
tab

gaia_edr3_sourceid,space,cluster,merged_cluster
int64,str8,int64,str4
634469512611911168,velocity,0,I
1277708416434956288,velocity,0,I
2798457105022823296,velocity,0,I
2831494371421184512,velocity,0,I
2910503176753011840,velocity,0,I
3454083549225619712,velocity,0,I
5154102081099625344,velocity,0,I
5365473158505801088,velocity,0,I
5413460450147569664,velocity,0,I


In [5]:
ascii.write(tab, 'clustering_result_tables/robust_cluster_source_id_mrt.txt', format='mrt')

In [12]:
ascii.write(tab[:5], 'clustering_result_tables/robust_cluster_source_id_stub.tex', format='latex',overwrite=True)